In [61]:
import pyspark
import findspark
from pyspark.sql import SparkSession


findspark.init()
sc = pyspark.SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-fb7f0f9d2079>:5 

In [66]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [63]:
dataRDD = sc.parallelize("Hello There! How are you?".split(" "))
dataRDD.collect()

['Hello', 'There!', 'How', 'are', 'you?']

In [69]:
txtFile = sc.textFile("/Users/Heath/Downloads/TitanicProblem/Titanic.txt")
txtFile.map(lambda line: line.upper()).collect()

['"1","1ST",1,"ALLEN, MISS ELISABETH WALTON",29.0000,"SOUTHAMPTON","ST LOUIS, MO","B-5","24160 L221","2","FEMALE"',
 '"2","1ST",0,"ALLISON, MISS HELEN LORAINE", 2.0000,"SOUTHAMPTON","MONTREAL, PQ / CHESTERVILLE, ON","C26","","","FEMALE"',
 '"3","1ST",0,"ALLISON, MR HUDSON JOSHUA CREIGHTON",30.0000,"SOUTHAMPTON","MONTREAL, PQ / CHESTERVILLE, ON","C26","","(135)","MALE"',
 '"4","1ST",0,"ALLISON, MRS HUDSON J.C. (BESSIE WALDO DANIELS)",25.0000,"SOUTHAMPTON","MONTREAL, PQ / CHESTERVILLE, ON","C26","","","FEMALE"',
 '"5","1ST",1,"ALLISON, MASTER HUDSON TREVOR", 0.9167,"SOUTHAMPTON","MONTREAL, PQ / CHESTERVILLE, ON","C22","","11","MALE"',
 '"6","1ST",1,"ANDERSON, MR HARRY",47.0000,"SOUTHAMPTON","NEW YORK, NY","E-12","","3","MALE"',
 '"7","1ST",1,"ANDREWS, MISS KORNELIA THEODOSIA",63.0000,"SOUTHAMPTON","HUDSON, NY","D-7","13502 L77","10","FEMALE"',
 '"8","1ST",0,"ANDREWS, MR THOMAS, JR",39.0000,"SOUTHAMPTON","BELFAST, NI","A-36","","","MALE"',
 '"9","1ST",1,"APPLETON, MRS EDWARD DALE (CHARLOT

In [70]:
spark = SparkSession.builder.appName('SparkByCynthia.com').getOrCreate()


In [117]:
df = spark.read.csv("/Users/Heath/Downloads/TitanicProblem/Titanic.txt").toDF("id","rank","alive","name","age","city","state","class","level","room_no","gender")
df.show()


+---+----+-----+--------------------+-------+-----------+--------------------+-------+-----------------+-------+------+
| id|rank|alive|                name|    age|       city|               state|  class|            level|room_no|gender|
+---+----+-----+--------------------+-------+-----------+--------------------+-------+-----------------+-------+------+
|  1| 1st|    1|Allen, Miss Elisa...|29.0000|Southampton|        St Louis, MO|    B-5|       24160 L221|      2|female|
|  2| 1st|    0|Allison, Miss Hel...| 2.0000|Southampton|Montreal, PQ / Ch...|    C26|             null|   null|female|
|  3| 1st|    0|Allison, Mr Hudso...|30.0000|Southampton|Montreal, PQ / Ch...|    C26|             null|  (135)|  male|
|  4| 1st|    0|Allison, Mrs Huds...|25.0000|Southampton|Montreal, PQ / Ch...|    C26|             null|   null|female|
|  5| 1st|    1|Allison, Master H...| 0.9167|Southampton|Montreal, PQ / Ch...|    C22|             null|     11|  male|
|  6| 1st|    1|  Anderson, Mr Harry|47.

In [118]:
df.createOrReplaceTempView("people")
sqldf = spark.sql("select * from people")
sqldf.show()

+---+----+-----+--------------------+-------+-----------+--------------------+-------+-----------------+-------+------+
| id|rank|alive|                name|    age|       city|               state|  class|            level|room_no|gender|
+---+----+-----+--------------------+-------+-----------+--------------------+-------+-----------------+-------+------+
|  1| 1st|    1|Allen, Miss Elisa...|29.0000|Southampton|        St Louis, MO|    B-5|       24160 L221|      2|female|
|  2| 1st|    0|Allison, Miss Hel...| 2.0000|Southampton|Montreal, PQ / Ch...|    C26|             null|   null|female|
|  3| 1st|    0|Allison, Mr Hudso...|30.0000|Southampton|Montreal, PQ / Ch...|    C26|             null|  (135)|  male|
|  4| 1st|    0|Allison, Mrs Huds...|25.0000|Southampton|Montreal, PQ / Ch...|    C26|             null|   null|female|
|  5| 1st|    1|Allison, Master H...| 0.9167|Southampton|Montreal, PQ / Ch...|    C22|             null|     11|  male|
|  6| 1st|    1|  Anderson, Mr Harry|47.

In [83]:
#1: Find the average age of people who died and who survived

sqldf = spark.sql("select alive, int(mean(age)) as avg_age from people group by alive")
sqldf.show()

+-----+-------+
|alive|avg_age|
+-----+-------+
|    0|     32|
|    1|     29|
+-----+-------+



In [113]:
#2: Number of males and females survived in following age range: (age <= 20), (20 < age <= 50) and (age > 50 and age = NA)

sqldf = spark.sql("select gender, ( CASE WHEN age <= 20 THEN 'lt20' WHEN age > 20 AND age <=50 THEN 'lt50' WHEN age > 50 THEN 'gt50' WHEN age = 'NA' THEN 'NA' ELSE '1' END) age_group, count(*) count from people group by gender, age_group")
sqldf.show()



+------+---------+-----+
|gender|age_group|count|
+------+---------+-----+
|  male|     lt20|   78|
|  male|       NA|  460|
|female|     lt50|  151|
|female|       NA|  220|
|female|     gt50|   25|
|  male|     lt50|  269|
|female|     lt20|   67|
|  male|     gt50|   43|
+------+---------+-----+



In [114]:
# 3. embarked locations and their count

sqldf = spark.sql("select city, count(*) from people group by city")
sqldf.show()

+-----------+--------+
|       city|count(1)|
+-----------+--------+
|       null|     492|
| Queenstown|      45|
|Southampton|     573|
|  Cherbourg|     203|
+-----------+--------+



In [141]:
# 4: Number of people survived in each class

#sqldf = spark.sql("select  (substr(class,1,1)) class_div ,   count(*) count from people where alive = 1 group by class_div")
sqldf = spark.sql("select  rank as class,   count(*) count from people where alive = 1 group by rank")

sqldf.show()

+-----+-----+
|class|count|
+-----+-----+
|  2nd|  119|
|  1st|  193|
|  3rd|  137|
+-----+-----+



In [146]:
# 5: Number of males survived whose age is less than 30 and travelling in 2nd class
sqldf = spark.sql("select gender,age ,  count(*) count from people where age < 30 and rank = '2nd' and gender = 'male' and alive = '1'  group by gender,age")
sqldf.show()

sqldf.createOrReplaceTempView("subset")
sqldf_subset = spark.sql("select * from subset")
sqldf_subset.show()


sqldf_subset_2 = spark.sql("select gender, sum(count) from subset group by gender ")
sqldf_subset_2.show()



+------+-------+-----+
|gender|    age|count|
+------+-------+-----+
|  male|22.0000|    1|
|  male|26.0000|    1|
|  male|24.0000|    1|
|  male|19.0000|    1|
|  male| 3.0000|    2|
|  male| 0.8333|    2|
|  male| 2.0000|    3|
|  male|20.0000|    1|
|  male| 8.0000|    2|
|  male| 1.0000|    2|
+------+-------+-----+

+------+-------+-----+
|gender|    age|count|
+------+-------+-----+
|  male|22.0000|    1|
|  male|26.0000|    1|
|  male|24.0000|    1|
|  male|19.0000|    1|
|  male| 3.0000|    2|
|  male| 0.8333|    2|
|  male| 2.0000|    3|
|  male|20.0000|    1|
|  male| 8.0000|    2|
|  male| 1.0000|    2|
+------+-------+-----+

+------+----------+
|gender|sum(count)|
+------+----------+
|  male|        16|
+------+----------+

